In [1]:
import pandas as pd
import numpy as np
import json

In [233]:
myFile = pd.read_csv('D:/project/550_Pattern/Project_1/550-p1-cset-krk-1.csv')
myFile
myFrame=pd.DataFrame(myFile.values, columns = ["White-King-File", "White-King-rank", "White-Rook-file", "White-Rook-rank","Black-King-file","Black-King-rank","Class"])
myFrame

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
0,c,1,e,2,d,3,draw
1,b,1,d,7,c,7,draw
2,d,2,g,1,g,2,draw
3,d,4,g,6,f,6,draw
4,c,1,b,5,a,5,draw
...,...,...,...,...,...,...,...
214,d,3,d,7,a,7,nine
215,b,1,b,4,g,1,nine
216,d,4,e,8,g,2,nine
217,c,2,d,5,f,1,nine


In [234]:
myFrame.shape

(219, 7)

In [235]:
# Split into training set:60%, holdout set:20%, Validation set: 20%

training_set = myFrame.sample(frac=0.6)
remaining_set = myFrame.drop(training_set.index)
training_set.reset_index(drop=True, inplace=True)

In [236]:
training_set

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
0,d,2,f,3,b,2,three
1,d,3,b,8,b,1,three
2,d,4,b,6,e,1,nine
3,c,3,f,4,b,1,two
4,d,1,b,6,b,1,seven
...,...,...,...,...,...,...,...
126,b,1,d,7,c,7,draw
127,c,3,f,2,a,1,three
128,c,2,a,8,a,1,zero
129,c,3,a,8,b,1,one


In [237]:
remaining_set.reset_index(drop=True, inplace=True)
remaining_set

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
0,c,1,e,2,d,3,draw
1,d,3,f,6,f,7,draw
2,b,1,h,3,g,2,draw
3,d,3,h,4,h,3,draw
4,d,4,e,1,d,1,draw
...,...,...,...,...,...,...,...
83,d,3,e,6,h,3,nine
84,d,4,d,8,g,1,nine
85,d,3,d,7,a,7,nine
86,b,1,b,4,g,1,nine


In [238]:
classes, cls_counts = np.unique(training_set['Class'], return_counts = True)
classes, cls_counts


(array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
        'three', 'two', 'zero'], dtype=object),
 array([13, 19, 11, 12, 14, 10, 10, 13, 13, 10,  6]))

In [239]:
def calculate_entropy(df_label):
    classes,class_counts = np.unique(df_label,return_counts = True)
    entropy_value = np.sum([(-class_counts[i]/np.sum(class_counts))*np.log2(class_counts[i]/np.sum(class_counts)) 
                        for i in range(len(classes))])
    return entropy_value

In [240]:
calculate_entropy(training_set['Class'])

3.4107386020233346

In [241]:

for cols in training_set.columns[:-1]:
    print(cols)

White-King-File
White-King-rank
White-Rook-file
White-Rook-rank
Black-King-file
Black-King-rank


In [242]:
# dataset = training_set
# feature = Feature or attribute of the dataset, i.e column name (training_set.columns[:-1]) E.g White-King-File
# label = Name of Class label colums, i.e. 'Class'

def calculate_information_gain(dataset,feature,label): 
    # Calculate the dataset entropy
    dataset_entropy = calculate_entropy(dataset[label])   
    values,feat_counts= np.unique(dataset[feature],return_counts=True)
    
    # Calculate the weighted feature entropy                                # Call the calculate_entropy function
    weighted_feature_entropy = np.sum([(feat_counts[i]/np.sum(feat_counts))*calculate_entropy(dataset.where(dataset[feature]
                              ==values[i]).dropna()[label]) for i in range(len(values))])    
    feature_info_gain = dataset_entropy - weighted_feature_entropy    
    return feature_info_gain

In [243]:
calculate_information_gain(training_set, 'White-King-File', 'Class')

0.4581138468066559

In [244]:
class DNode:
    def __init__(self, attribute, values):
        self.attribute = attribute
        self.values = values
        self.children = {}
    

In [245]:
class LeafNode:
    def __init__(self,value):
        self.value = value
        

In [246]:
def copy_list(l):
    return l[:]

In [247]:
attributes = training_set.columns[:-1].tolist()
class_label = 'Class'

In [248]:
def select_attribute(dataset, attributes, class_label):
    
    gains = [calculate_information_gain(dataset, attribute, class_label) for attribute in attributes]
#     print("gains",gains)
    maxGainIndex = np.argmax(gains)
    maxGainValue = gains[maxGainIndex]
#     print("maxGainIndex",maxGainIndex)
#     print("maxGainValue",maxGainValue)
#     print("highGainAttribute:",attributes[maxGainIndex])
    
    
    
    if maxGainValue == 0.0:
        value = np.unique(dataset[class_label])[0]
#         print('value',value)
        return LeafNode(value)
    
    else:
        newAttribute = attributes[maxGainIndex]
        values = np.unique(dataset[newAttribute])
#         print("newAttribute:",newAttribute)
#         print("values:",values)
        return DNode(newAttribute, values)
        

In [249]:
select_attribute(training_set, attributes, class_label)

In [250]:
def create_dtree(dataset, attributes, class_label):        
    node = select_attribute(dataset, attributes, class_label)
    
    if isinstance(node, DNode):
        newDataSet = dataset.drop(columns = node.attribute)        
        newAttributes = copy_list(attributes)
        newAttributes.remove(node.attribute)        
#         print('newAttributes', newAttributes)
#         print('node.values',node.values)
        for v in node.values:
#             print("v:",v)
            node.children[v] = create_dtree(dataset.where(dataset[node.attribute] == v).dropna(), newAttributes, class_label)
    
    return node

In [251]:
rootNode = create_dtree(training_set, copy_list(attributes), class_label)

In [252]:
def check_treeRec(node):
    if isinstance(node, LeafNode):
        print('leaf node value:', node.value)
        return 
    
    for k,v in node.children.items():
        check_treeRec(v)
    
    return

In [253]:
check_treeRec(rootNode)

leaf node value: zero
leaf node value: six
leaf node value: three
leaf node value: one
leaf node value: three
leaf node value: four
leaf node value: eight
leaf node value: draw
leaf node value: eight
leaf node value: seven
leaf node value: seven
leaf node value: two
leaf node value: six
leaf node value: one
leaf node value: four
leaf node value: one
leaf node value: two
leaf node value: four
leaf node value: four
leaf node value: five
leaf node value: four
leaf node value: five
leaf node value: seven
leaf node value: three
leaf node value: draw
leaf node value: seven
leaf node value: five
leaf node value: three
leaf node value: two
leaf node value: six
leaf node value: one
leaf node value: one
leaf node value: five
leaf node value: one
leaf node value: five
leaf node value: seven
leaf node value: three
leaf node value: eight
leaf node value: three
leaf node value: five
leaf node value: four
leaf node value: five
leaf node value: four
leaf node value: three
leaf node value: two
leaf nod

In [254]:
def predict(node, row, correctAns):
    if isinstance(node, LeafNode):
        return node.value == correctAns
    print('Attr:',node.attribute,", value:", row[node.attribute])
    return predict(node.children[row[node.attribute]], row, correctAns)
    

In [255]:
ans = predict(rootNode, training_set.loc[4], 'four')
print('ans:',ans)

Attr: Black-King-file , value: b
Attr: White-Rook-file , value: b
Attr: White-King-rank , value: 1
ans: False
